# Parquet, DuckDB, and Arrow: Columnar data examples

Let's start by accessing an existing 1.3 GB parquet file over the web.

In [1]:
# Set up imports
import duckdb

We'll be accessing a file from the ADC. Here's the path. First, read it in with duckdb, and inspect the columns.

In [2]:
iwp_path = 'https://arcticdata.io/data/10.18739/A24F1MK7Q/iwp_geotiff_low_medium/raster_summary.parquet'

In [3]:
# Read parquet, and show the columns
iwp = duckdb.read_parquet(iwp_path)

In [4]:
print(iwp.columns)

['stat', 'bounds', 'min', 'max', 'mean', 'median', 'std', 'var', 'sum', 'path', 'tile', 'z']


In [5]:
# Use duckdb to count the rows -- a metadata query
duckdb.sql("SELECT count(*) as n from iwp").show()

┌──────────┐
│    n     │
│  int64   │
├──────────┤
│ 18150329 │
└──────────┘



In [7]:
duckdb.sql("select distinct stat from iwp order by stat;").show()

┌──────────────┐
│     stat     │
│   varchar    │
├──────────────┤
│ iwp_coverage │
└──────────────┘



In [8]:
low_coverage = iwp.project("bounds, sum").filter("sum < 10")
low_coverage.count("*")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        37169 │
└──────────────┘

In [9]:
import os
low_coverage.write_parquet("low_coverage.parquet")
os.stat("low_coverage.parquet").st_size/(1024*1024)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

1.4038171768188477

## Delta Fisheries using Arrow

In this example, we'll read in a dataset of fish abundance in the San Francisco Estuary, which is published in csv format on the [Environmental Data Initiative](https://portal.edirepository.org/nis/mapbrowse?scope=edi&identifier=1075&revision=1). This dataset isn't huge, but it is big enough (3 GB) that working with it locally can be fairly taxing on memory. Motivated by user difficulties in actually working with the data, the [`deltafish` R](https://github.com/Delta-Stewardship-Council/deltafish) package was written using the R implementation of `arrow`. It works by downloading the EDI repository data, writing it to a local cache in parquet format, and using `arrow` to query it. In this example, I've put the Parquet files in a sharable location so we can explore them using `pyarrow`.


In [1]:
import pyarrow.dataset as ds
import numpy as np
import pandas as pd

In [ ]:
# Create an arrow Dataset from a locally saved parquet file
